In [1]:
import numpy as np
from numpy import linalg as la

def compute_transition_probabilities(M: int, dt: float, r: float, q: float, sigma: float):
    """Calculate the transition probabilities for the finite difference matrix."""
    j = np.arange(0, M, dtype=np.float64)
    a = 0.5 * (r - q) * j * dt - 0.5 * sigma**2 * j**2 * dt
    b = 1.0 + sigma**2 * j**2 * dt + r * dt
    c = -0.5 * (r - q) * j * dt - 0.5 * sigma**2 * j**2 * dt
    return a, b, c

def create_tridiagonal_matrix(a: np.ndarray, b: np.ndarray, c: np.ndarray):
    """Construct and invert the tridiagonal matrix from transition probabilities."""
    M = len(b)
    A = np.diag(b) + np.diag(a[1:], k=-1) + np.diag(c[:-1], k=1)
    B = la.inv(A)
    return B, la.norm(B, np.inf)

def initialize_option_matrix(N: int, M: int, S_max: float, dS: float, K: float, r: float, dt: float, is_call: bool):
    """Set up the initial and boundary conditions for the option pricing matrix."""
    FV = np.zeros((N + 1, M + 1))
    if is_call:
        FV[:, 0] = 0.0
        FV[:, M] = S_max * np.exp(-r * (N - np.arange(N + 1)) * dt)
        FV[N, :] = np.maximum(np.arange(0, S_max + dS, dS) - K, 0)
    else:
        FV[:, 0] = K * np.exp(-r * (N - np.arange(N + 1)) * dt)
        FV[:, M] = 0.0
        FV[N, :] = np.maximum(K - np.arange(0, S_max + dS, dS), 0)
    return FV

def backward_iteration(FV: np.ndarray, B: np.ndarray, a: np.ndarray, c: np.ndarray, N: int, M: int, dS: float, K: float, is_call: bool, is_american: bool):
    """Perform backward iteration to calculate the option price at each time step."""
    for i in range(N - 1, -1, -1):
        k_i = np.zeros(M)
        k_i[0] = -a[0] * FV[i + 1, 0]
        k_i[M - 1] = -c[M - 1] * FV[i + 1, M]
        
        FV[i, :M] = np.dot(FV[i + 1, :M] + k_i, B)

        if is_american:
            intrinsic_value = np.maximum(np.arange(0, M + 1) * dS - K if is_call else K - np.arange(0, M + 1) * dS, FV[i, :])
            FV[i, :] = intrinsic_value
    return FV

def vanilla_option_implicit(S_0: float, T: float, K: float, r: float, q: float, sigma: float, N: int, M: int, is_call: bool, is_american: bool) -> np.ndarray:
    """Calculate the fair value of a vanilla European or American call/put option."""
    results = np.zeros((2, 1), dtype=float)
    S_max = 2.0 * K
    dS = S_max / M
    dt = T / N

    # Compute transition probabilities
    a, b, c = compute_transition_probabilities(M, dt, r, q, sigma)
    
    # Create tridiagonal matrix and its inverse
    B, infinity_norm = create_tridiagonal_matrix(a, b, c)
    results[1] = infinity_norm

    # Initialize option price matrix with boundary conditions
    FV = initialize_option_matrix(N, M, S_max, dS, K, r, dt, is_call)

    # Perform backward iteration
    FV = backward_iteration(FV, B, a, c, N, M, dS, K, is_call, is_american)
    
    # Store fair value of option in results
    results[0] = FV[0, M // 2]
    return results

# Sample test execution
if __name__ == "__main__":
    S_0 = 50.0
    T = 1.0
    K = 50.0
    r = 0.1
    q = 0.00
    sigma = 0.25
    N = 100
    M = 40
    is_call = True
    is_american = False
    print(vanilla_option_implicit(S_0, T, K, r, q, sigma, N, M, is_call, is_american))


[[5.17924845]
 [0.99937502]]


In [32]:
import unittest

# Define the utility functions and main function here (or import them if they are in a module)

# Define the Test Cases
class TestVanillaOptionImplicit(unittest.TestCase):
    # Test methods go here as defined previously
    def test_compute_transition_probabilities(self):
        M, dt, r, q, sigma = 40, 0.01, 0.1, 0.02, 0.25
        a, b, c = compute_transition_probabilities(M, dt, r, q, sigma)
        self.assertEqual(len(a), M)
        self.assertEqual(len(b), M)
        self.assertEqual(len(c), M)
        self.assertAlmostEqual(a[1], 8.750000000000001e-05, places=4)
        self.assertAlmostEqual(b[1], 1.001625, places=4)
        self.assertAlmostEqual(c[1], -0.0007125, places=4)

    def test_create_tridiagonal_matrix(self):
        M = 40
        a = np.full(M, -0.5)
        b = np.full(M, 2.0)
        c = np.full(M, -0.5)
        B, infinity_norm = create_tridiagonal_matrix(a, b, c)
        self.assertEqual(B.shape, (M, M))
        self.assertGreater(infinity_norm, 0)
        self.assertTrue(np.allclose(B @ (np.diag(b) + np.diag(a[1:], k=-1) + np.diag(c[:-1], k=1)), np.eye(M)))

    def test_initialize_option_matrix(self):
        N, M = 100, 40
        S_max, K, r, dt = 100, 50, 0.1, 0.01
        dS = S_max / M
        FV_call = initialize_option_matrix(N, M, S_max, dS, K, r, dt, is_call=True)
        self.assertEqual(FV_call.shape, (N + 1, M + 1))
        self.assertAlmostEqual(FV_call[N, 0], 0.0)
        self.assertTrue(np.all(FV_call[N, :] >= 0))
        FV_put = initialize_option_matrix(N, M, S_max, dS, K, r, dt, is_call=False)
        self.assertEqual(FV_put.shape, (N + 1, M + 1))
        self.assertAlmostEqual(FV_put[N, M], 0.0)
        self.assertTrue(np.all(FV_put[N, :] >= 0))

    def test_backward_iteration(self):
        N, M = 5, 5
        S_max, K, dS = 100, 50, 20
        r, dt = 0.1, 0.01
        is_call = True
        is_american = False
        a = np.full(M, -0.5)
        b = np.full(M, 2.0)
        c = np.full(M, -0.5)
        B, _ = create_tridiagonal_matrix(a, b, c)
        FV = initialize_option_matrix(N, M, S_max, dS, K, r, dt, is_call)
        FV_result = backward_iteration(FV, B, a, c, N, M, dS, K, is_call, is_american)
        self.assertEqual(FV_result.shape, (N + 1, M + 1))
        self.assertTrue(np.all(FV_result >= 0))

    def test_vanilla_option_implicit(self):
        S_0, T, K, r, q, sigma, N, M = 50.0, 1.0, 50.0, 0.1, 0.0, 0.25, 100, 40
        is_call, is_american = True, False
        result = vanilla_option_implicit(S_0, T, K, r, q, sigma, N, M, is_call, is_american)
        self.assertEqual(result.shape, (2, 1))
        option_value = result[0, 0]
        self.assertGreater(option_value, 0)
        self.assertLess(option_value, S_0)


In [33]:
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestVanillaOptionImplicit))

.....
----------------------------------------------------------------------
Ran 5 tests in 0.010s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>